In [17]:
import pdfplumber
from pdfminer.pdfpage import PDFPage
from polyglot.detect import Detector
import os 
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")
from tqdm import tqdm as tqdm 
from pdfminer.pdfdocument import PDFSyntaxError

In [10]:
def scan_check(filename): 
    with open(filename, 'rb') as infile:
      for page in PDFPage.get_pages(infile):
            if 'Font' in page.resources.keys():
                is_scanned = False
            else:
                is_scanned = True
    return is_scanned

In [1]:
def pdf_to_txt(filename):
  if not scan_check(filename):
    with open(f"{os.path.splitext(filename)[0]}.txt", 'w') as txt_file: # Modified 
        pdf = pdfplumber.open(filename)
        for i in range(0, len(pdf.pages)):
          text = pdf.pages[i].extract_text()
          try:
              detector = Detector(text)
              if detector.language.name == 'Azerbaijani':
                    txt_file.write(text)
          except:
              pass       

       

In [23]:

# loop through all the PDF files in the folder

def process_pdf(file_path):
    try:
        # pass the file to the processing function
        pdf_to_txt(file_path)
    except PDFSyntaxError:
        # handle the "No /Root object" error
        print(f"{file_path}: No /Root object! - Is this really a PDF?")
    except TypeError:
        print(f"{file_path}: Cannot convert 'PSKeyword' object to bytearray")

pdf_folder = ''
count = 0

# iterate through all the files in the folder
for filename in tqdm(os.listdir(pdf_folder)):
    if filename.endswith('.pdf'):
        # construct the full file path
        file_path = os.path.join(pdf_folder, filename)
        # process the file
        process_pdf(file_path)
        count += 1

print(f"{count} PDFs were processed")